In [1]:
import os,sys,glob
import numpy as np
import json
import pandas as pd
# Move two levels up work from project level, not location of analysis file
os.chdir('../..')
print(os.getcwd())

recollect = False

d:\Users\rschoete\Documents\Optogenetics\OptogeneticExcitability\Code


In [2]:

filepath = './Results/SDC/SDC_CA1PCcACsig5_invivoUgent'
savename = os.path.join(filepath,'all_data.csv')

# list parameters of interest
positional_input = ['x','y','z','phi','theta','psi']
cell_init_options = ['phi_0','theta_0','psi_0']
settings_options = ['seed','celsius','dt']
opsin_options = ['Gmax','distribution','opsinmech','distribution_method','opsinlocations']
field_options = ['field','dc','nPulse','prf']
result = ['amp','dur']
all_columns = result+field_options+opsin_options+cell_init_options+positional_input

# opsinlocations to single word
opsinLocation_map = {'1000':'soma','0100':'axon','0010':'apic','0001':'dend','0011':'alldend','1111':'allsec'}
if recollect:
    # list all data and input file locations
    alldata_files = [x for x in glob.glob(os.path.join(filepath,'**/data.json'),recursive=True) if not 'ToConcat' in x]
    allinput_files = [x for x in glob.glob(os.path.join(filepath,'**/input.json'),recursive=True) if not 'ToConcat' in x]
    dir_list = [x for x in glob.glob(os.path.join(filepath,'*')) if os.path.isdir(x) and not 'ToConcat' in x]
    print(f"result files: {len(alldata_files)}, input files: {len(allinput_files)}, directories: {len(dir_list)}")

    # init master_df: dataframe where all results will be stored
    master_df = pd.DataFrame(columns=all_columns)

    for data_path,input_path in zip(alldata_files,allinput_files):
        #print(data_path,input_path,)
        #Load data and input file
        with open(data_path,'r') as f:
            mydata = json.load(f)
        with open(input_path,'r') as f:
            myinput = json.load(f)

        # store global input info
        cell_init_dict = {key: myinput['info']['settings']['cellsopt']['init_options'][key.rsplit('_',1)[0]] for key in cell_init_options}
        settings_dict = {key: myinput['info']['settings'][key] for key in settings_options}
        opsin_dict = {**{opsin_options[0]:myinput['info']['settings']['cellsopt']['opsin_options'][opsin_options[0]+'_total']},**{key: myinput['info']['settings']['cellsopt']['opsin_options'][key] for key in opsin_options[1:]}}
        field_dict = {field_options[0]:myinput['info']['settings']['stimopt']['Ostimparams']['filepath'].rsplit('/',1)[-1].rsplit('.txt',1)[0],
        field_options[1]:myinput['info']['settings']['analysesopt']['SDOptogenx']['options'][field_options[1]+'_sdc'],
        field_options[2]:myinput['info']['settings']['analysesopt']['SDOptogenx']['r_p'.join(field_options[2].split('P'))+'OI'],
        field_options[3]:myinput['info']['settings']['stimopt']['Ostimparams']['options'][field_options[3]]}

        opsin_dict['opsinlocations'] = opsinLocation_map[''.join([str(int(any([loc in oloc for oloc in opsin_dict['opsinlocations']]))) for loc in list(opsinLocation_map.values())[:4]])]

        info_dict = {**field_dict,**opsin_dict,**cell_init_dict,**settings_dict}

        # store simulation specific info and add to dataframe
        durs = myinput["info"]['settings']['analysesopt']['SDOptogenx']["durs"]
        for k in mydata.keys():
            cellPos_dict = {**{key:myinput[k]['xT'][i]for i,key in enumerate(positional_input[0:3])},**{key:myinput[k][key]for key in positional_input[3:]}}
            for amp,dur in zip(mydata[k]["SDcurve"]["Optogenx"],durs):
                result_dict = {result[0]:amp, result[1]:dur}
                intm = pd.DataFrame({**result_dict,**cellPos_dict,**info_dict},index=[0],columns=all_columns)
                master_df = pd.concat([master_df,pd.DataFrame({**result_dict,**cellPos_dict,**info_dict},index=[0],columns=all_columns)])

    master_df = master_df.reset_index(drop=True)
    master_df.to_csv(savename)
else:
    master_df = pd.read_csv(savename)

result files: 160, input files: 160, directories: 160
./Results/SDC/SDC_CA1PCcACsig5_invivoUgent\CA1_PC_cAC_sig5_all_np1_0202209302151\data.json ./Results/SDC/SDC_CA1PCcACsig5_invivoUgent\CA1_PC_cAC_sig5_all_np1_0202209302151\input.json
./Results/SDC/SDC_CA1PCcACsig5_invivoUgent\CA1_PC_cAC_sig5_all_np1_1202209302151\data.json ./Results/SDC/SDC_CA1PCcACsig5_invivoUgent\CA1_PC_cAC_sig5_all_np1_1202209302151\input.json
./Results/SDC/SDC_CA1PCcACsig5_invivoUgent\CA1_PC_cAC_sig5_all_np1_2202209302151\data.json ./Results/SDC/SDC_CA1PCcACsig5_invivoUgent\CA1_PC_cAC_sig5_all_np1_2202209302151\input.json
./Results/SDC/SDC_CA1PCcACsig5_invivoUgent\CA1_PC_cAC_sig5_all_np1_3202209302151\data.json ./Results/SDC/SDC_CA1PCcACsig5_invivoUgent\CA1_PC_cAC_sig5_all_np1_3202209302151\input.json
./Results/SDC/SDC_CA1PCcACsig5_invivoUgent\CA1_PC_cAC_sig5_all_np1_4202209302151\data.json ./Results/SDC/SDC_CA1PCcACsig5_invivoUgent\CA1_PC_cAC_sig5_all_np1_4202209302151\input.json
./Results/SDC/SDC_CA1PCcACsig5_

,amp,dur,field,dc,nPulse,prf,Gmax,distribution,opsinmech,distribution_method,opsinlocations,phi_0,theta_0,psi_0,x,y,z,phi,theta,psi
0,69062.5,0.01,Ugent470nIrr_np1e7_res5emin3_gf1_cyl_5x10,1,1,0.1,2.160768,distribution = lambda x: 10.0,chr2h134r,3d,allsec,0,-1.570796,0,0.0,0.0,0.0,0,-1.570796,0
1,1856.0547,0.1,Ugent470nIrr_np1e7_res5emin3_gf1_cyl_5x10,1,1,0.1,2.160768,distribution = lambda x: 10.0,chr2h134r,3d,allsec,0,-1.570796,0,0.0,0.0,0.0,0,-1.570796,0
2,237.807,1,Ugent470nIrr_np1e7_res5emin3_gf1_cyl_5x10,1,1,0.1,2.160768,distribution = lambda x: 10.0,chr2h134r,3d,allsec,0,-1.570796,0,0.0,0.0,0.0,0,-1.570796,0
3,94.0081,10,Ugent470nIrr_np1e7_res5emin3_gf1_cyl_5x10,1,1,0.1,2.160768,distribution = lambda x: 10.0,chr2h134r,3d,allsec,0,-1.570796,0,0.0,0.0,0.0,0,-1.570796,0
4,72.1953,100,Ugent470nIrr_np1e7_res5emin3_gf1_cyl_5x10,1,1,0.1,2.160768,distribution = lambda x: 10.0,chr2h134r,3d,allsec,0,-1.570796,0,0.0,0.0,0.0,0,-1.570796,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67755,NaN,0.01,Ugent470nIrr_np1e7_res5emin3_gf1_cyl_5x10,0.01,2,0.001,2.211878,distribution = lambda x: 316.22776601683796,chr2h134r,3d,soma,0,-1.570796,0,0.0,800.0,4500.0,0,-1.570796,0
67756,NaN,0.1,Ugent470nIrr_np1e7_res5emin3_gf1_cyl_5x10,0.01,2,0.001,2.211878,distribution = lambda x: 316.22776601683796,chr2h134r,3d,soma,0,-1.570796,0,0.0,800.0,4500.0,0,-1.570796,0
67757,9156250.0,1,Ugent470nIrr_np1e7_res5emin3_gf1_cyl_5x10,0.01,2,0.001,2.211878,distribution = lambda x: 316.22776601683796,chr2h134r,3d,soma,0,-1.570796,0,0.0,800.0,4500.0,0,-1.570796,0
67758,2396362.3047,10,Ugent470nIrr_np1e7_res5emin3_gf1_cyl_5x10,0.01,2,0.001,2.211878,distribution = lambda x: 316.22776601683796,chr2h134r,3d,soma,0,-1.570796,0,0.0,800.0,4500.0,0,-1.570796,0
